In [ ]:
%pip install datasets
%pip install py7zr
%pip install tabulate
%pip install numpy

In [ ]:
from datasets import load_dataset
import json
from tabulate import tabulate
import numpy as np

# Load ACU Datasets

In [ ]:
cnndm_test = load_dataset("Salesforce/rose", "cnndm_test")["data"]
cnndm_val = load_dataset("Salesforce/rose", "cnndm_validation")["data"]
xsum = load_dataset("Salesforce/rose", "xsum")["data"]
samsum = load_dataset("Salesforce/rose", "samsum")["data"]
cnndm_protocol = load_dataset("Salesforce/rose", "cnndm_protocol")["data"]
cnndm_protocol_gpt3 = load_dataset("Salesforce/rose", "cnndm_protocol_gpt3")["data"]

# Human Annotations with ACU Protocol
## Check the dataset information of the ACU annotations on CNNDM

In [7]:
print(cnndm_test)

Dataset({
    features: ['source', 'reference', 'reference_acus', 'count_id', 'example_id', 'annotations', 'system_outputs'],
    num_rows: 500
})


## Check an example of ACU annotations

In [8]:
print(json.dumps(cnndm_test[0], indent=2))

{
  "source": "Club Tijuana star Juan Arango conjured memories Luis Suarez in his team's 4-3 defeat by Monterrey in the Mexican league - but it was not through prodigious scoring. The Venezuelan icon Arango sank his teeth into the shoulder of Jesus Zavela as his temper flared in the defeat. He was not booked by the referee but could face a heavy retrospective ban. Juan Arango (left) bites the shoulder of opponent Jesus Zavela in a moment of madness . Zavala holds his shoulder after being bitten by Arango, in the game Zavala's side won 4-3 in Mexico . Zavala shows the referee the mark on his shoulder after being bittern by Arango . Arango (right) earlier scored a magnificent free kick to bring his Club Tijuana team level against Monterrey . Arango had earlier curled in a magnificent free kick for his team to bring them level after falling 2-0 down early on in the encounter. But the 34-year-old overshadowed his goal with the bite as television cameras picked up the moment of madness. Ara

## Check the available systems

In [9]:
print(cnndm_test[0]["annotations"].keys())

dict_keys(['bart', 'gold', 'pegasus', 'brio', 'gsum', 'simcls', 'cliff', 'ctrlsum', 'frost', 'glob', 'matchsum', 'brio-ext'])


## Check the available annotations 

In [10]:
print(cnndm_test[0]["annotations"]["brio"].keys())

dict_keys(['acu_labels', 'acu', 'normalized_acu'])


## Calculate the average ACU scores of each system

In [11]:
systems = list(cnndm_test[0]["annotations"].keys())
system_scores = [(system, np.mean([x["annotations"][system]["acu"] for x in cnndm_test]).item()) for system in systems]
system_scores = sorted(system_scores, key=lambda x: x[1], reverse=True)
print(tabulate(system_scores, headers=["System", "ACU Score"]))

System      ACU Score
--------  -----------
ctrlsum      0.445805
gsum         0.444687
brio         0.440345
matchsum     0.425005
brio-ext     0.417233
simcls       0.404669
bart         0.38832
cliff        0.385072
frost        0.384386
gold         0.381019
pegasus      0.375578
glob         0.364016


## Calculate the average *normalized* ACU scores of each system

In [12]:
systems = list(cnndm_test[0]["annotations"].keys())
system_scores = [(system, np.mean([x["annotations"][system]["normalized_acu"] for x in cnndm_test]).item()) for system in systems]
system_scores = sorted(system_scores, key=lambda x: x[1], reverse=True)
print(tabulate(system_scores, headers=["System", "ACU Score"]))

System      ACU Score
--------  -----------
brio         0.371826
ctrlsum      0.36131
simcls       0.359992
gsum         0.348765
glob         0.340702
gold         0.33799
frost        0.336847
matchsum     0.336829
brio-ext     0.335767
cliff        0.329551
bart         0.323364
pegasus      0.319942


# Human Annotation with Four Different Protocols
## Check available systems

In [13]:
print(cnndm_protocol[0]["annotations"].keys())

dict_keys(['bart', 'gold', 'pegasus', 'brio', 'gsum', 'simcls', 'cliff', 'ctrlsum', 'frost', 'glob', 'matchsum', 'brio-ext'])


## Check available protocols

In [14]:
print(cnndm_protocol[0]["annotations"]["brio"].keys())

dict_keys(['prior', 'ref_based', 'ref_free', 'acu_labels', 'acu', 'normalized_acu'])


## Check GPT-3 performance under different protocols

In [15]:
protocols = ["prior", "ref_based", "ref_free", "acu"]
result = [np.mean([x["annotations"]["gpt3"][p] for x in cnndm_protocol_gpt3]).item() for p in protocols]
print(tabulate([result], headers=protocols))

  prior    ref_based    ref_free       acu
-------  -----------  ----------  --------
3.72333         2.74     3.76333  0.268991


## Calculate *LLMs* performance under different protocols (with baseline fine-tuned models)

In [16]:
protocols = ["prior", "ref_based", "ref_free", "acu"]
systems = ["gpt3", "brio", "t0", "bart"]
result = [[s]+ [np.mean([x["annotations"][s][p] for x in cnndm_protocol_gpt3]).item() for p in protocols] for s in systems]
print(tabulate(result, headers=["System"]+protocols))

System      prior    ref_based    ref_free       acu
--------  -------  -----------  ----------  --------
gpt3      3.72333      2.74        3.76333  0.268991
brio      3.51333      3.07333     3.49     0.429045
t0        3.33333      2.84333     3.24333  0.294712
bart      3.58333      2.92667     3.52333  0.36711


## Calculate *fine-tuned* system performance under different protocols

In [17]:
protocols = ["prior", "ref_based", "ref_free", "acu"]
systems = list(cnndm_protocol[0]["annotations"].keys())
result = [[s]+ [np.mean([x["annotations"][s][p] for x in cnndm_protocol]).item() for p in protocols] for s in systems]
print(tabulate(result, headers=["System"]+protocols))

System      prior    ref_based    ref_free       acu
--------  -------  -----------  ----------  --------
bart         3.12         2.47        3.4   0.36711
gold         2.74         2.52        2.98  0.370999
pegasus      2.74         2.44        2.99  0.378522
brio         3.09         2.52        3.29  0.429045
gsum         3.3          2.54        3.53  0.432917
simcls       2.93         2.53        3.18  0.397583
cliff        2.92         2.49        3.3   0.393997
ctrlsum      3.3          2.56        3.26  0.451103
frost        2.79         2.58        2.96  0.404047
glob         2.79         2.55        2.91  0.349981
matchsum     3.05         2.36        3.38  0.396766
brio-ext     3.1          2.38        3.31  0.370685
